In [1]:
# vision transformer for mfcc

In [1]:
import librosa
from torch import nn
import torch
from torchvision import datasets, transforms
from torch.utils.data import Dataset
import os
import soundfile
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader, random_split
from PIL import Image
torch.random.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
dataset = datasets.ImageFolder(
    root='./data/spectrograms/binary',
    transform=transforms.Compose([transforms.Resize((224,224)),
                                  transforms.ToTensor()
                                  ])
)
print(dataset)

Dataset ImageFolder
    Number of datapoints: 4752
    Root location: ./data/spectrograms/binary
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )


In [ ]:
from torch.utils.data import Subset
class_counts = {}
for _, label in dataset:
    label = label
    if label not in class_counts:
        class_counts[label] = 0
    class_counts[label] += 1

# Calculate the desired number of samples for each class in training and validation sets
total_samples = len(dataset)
train_ratio = 0.8  # Adjust as needed
train_class_counts = {label: int(train_ratio * count) for label, count in class_counts.items()}
val_class_counts = {label: count - train_class_counts[label] for label, count in class_counts.items()}

# Create samplers for training and validation sets while maintaining class balance
train_indices = []
val_indices = []
shuffled_dataset = DataLoader(dataset, shuffle=True).dataset
for idx, (_, label) in enumerate(shuffled_dataset):
    if train_class_counts[label] > 0:
        train_indices.append(idx)
        train_class_counts[label] -= 1
    else:
        val_indices.append(idx)

train_dataset = Subset(shuffled_dataset, train_indices)
validation_dataset = Subset(shuffled_dataset, val_indices)

In [3]:
train_dataset, validation_dataset = random_split(dataset, [0.8, 0.2])

In [4]:
len(train_dataset), len(validation_dataset)

(3802, 950)

In [5]:
train_dataloader = DataLoader(train_dataset, batch_size = 64, shuffle = True)
validation_dataloader = DataLoader(validation_dataset, batch_size = 64)

In [6]:
from torchvision.models import vit_b_16

In [7]:
model = vit_b_16(pretrained = False)
model.heads.head = nn.Linear(in_features = 768, out_features = len(dataset.classes))
model

c:\Users\mipec\CUDA_kernel\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\mipec\CUDA_kernel\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_a

In [20]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
model.to(device)
num_epochs = 1  # Adjust as needed
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    total_loss = 0.0

    for batch_idx, (images, labels) in enumerate(train_dataloader):
        optimizer.zero_grad()  # Zero out gradients
        outputs = model(images.to(device))
        loss = criterion(outputs, labels.to(device)) 
        loss.backward()  # Backpropagate gradients
        optimizer.step()  # Update model parameters

        total_loss += loss.item()

        if batch_idx % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}] Batch [{batch_idx+1}/{len(train_dataloader)}] Loss: {loss.item():.4f}", end="\r")

    average_loss = total_loss / len(train_dataloader)
    acc = 0
    for images, labels in validation_dataloader:
        model.eval()
        acc += (torch.max(model(images.to(device)).cpu(), 1)[1] == labels).sum().item()
    acc /= len(validation_dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}] Average Loss: {average_loss:.4f}, val_accuracy: {acc}")

Epoch [1/1] Average Loss: 0.6848, val_accuracy: 0.5463157894736842


In [21]:
torch.save(model.state_dict(), 'vit_binary_epoch2.pth')

### 0.608 acc after 1 epoch

### 2 epoch avg loss 0.6848, val_accuracy: 0.5463157894736842

In [8]:
model.load_state_dict(torch.load('vit_binary_epoch1.pth'))
model.to(device)

VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_a